In [1]:
!pip install contractions

In [2]:
# Import NLTK and Spacy Libraries
import string, re, nltk
from string import punctuation
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
import spacy
import contractions
import pandas as pd

In [3]:
# Importing the tensorflow libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.utils import to_categorical

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [5]:
nltk.download("all")
!python -m spacy download en_core_web_sm

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\gupta\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\gupta\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\gupta\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\gupta\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\gupta\AppData\Roaming\nltk_data...
[nltk_data]    | 

     ---------------------------------------- 12.8/12.8 MB 9.5 MB/s eta 0:00:00


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
tweets = pd.read_csv("nlp text daqta.csv",encoding='latin-1',header=None)
tweets

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1497,0,1468167571,Tue Apr 07 00:05:02 PDT 2009,NO_QUERY,SilkCharm,@kcarruthers I'm only 540 years old in pixie y...
1498,0,1468167936,Tue Apr 07 00:05:09 PDT 2009,NO_QUERY,dontlookaway,body clock still up the chuffer..note never ev...
1499,0,1468167941,Tue Apr 07 00:05:08 PDT 2009,NO_QUERY,racadwall2,stupid thing wont let me get into my old twitt...
1500,0,1468168034,Tue Apr 07 00:05:12 PDT 2009,NO_QUERY,RopeMarksMuse,I am soooo tired @ work


In [7]:
tweets.columns = ["target","ids","date","flag","user","text"]
tweets

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1497,0,1468167571,Tue Apr 07 00:05:02 PDT 2009,NO_QUERY,SilkCharm,@kcarruthers I'm only 540 years old in pixie y...
1498,0,1468167936,Tue Apr 07 00:05:09 PDT 2009,NO_QUERY,dontlookaway,body clock still up the chuffer..note never ev...
1499,0,1468167941,Tue Apr 07 00:05:08 PDT 2009,NO_QUERY,racadwall2,stupid thing wont let me get into my old twitt...
1500,0,1468168034,Tue Apr 07 00:05:12 PDT 2009,NO_QUERY,RopeMarksMuse,I am soooo tired @ work


In [8]:
# Removing the uncessary columns
tweets.drop(["ids","date","flag","user"],axis=1,inplace=True)

In [9]:
tweets

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1497,0,@kcarruthers I'm only 540 years old in pixie y...
1498,0,body clock still up the chuffer..note never ev...
1499,0,stupid thing wont let me get into my old twitt...
1500,0,I am soooo tired @ work


In [10]:
len(tweets)

1502

***Text Preprocessing***

In [11]:
# Checking for duplicates and null values
tweets.dropna(inplace=True)
tweets.drop_duplicates(inplace = True)
tweets.reset_index(drop=True,inplace=True)

In [12]:
len(tweets)

1502

In [13]:
#Regular Expression
regexp = RegexpTokenizer("[\w']+")

In [14]:
# Lowecase the text
def lower_text(text):
  text = text.lower()
  return text

In [15]:
# Remove spaces from the text
def remove_spaces(text):
  text = text.strip()
  return text

In [16]:
def remove_punctuation(text):
    punct = string.punctuation
    punct = punct.replace("'", "")  # Removing single quote from punctuation
    text = text.translate(str.maketrans("", "", punct))
    return text

In [17]:
def remove_html(text):
  html = re.compile(r'<*?>')
  text = html.sub(r'',text)
  return text

In [18]:
# Removing emojis
def remove_emoji(text):
  emoji_pattern = re.compile("["
  u"\U0001F600-\U0001F64F" # emoticons
  u"\U0001F300-\U0001F5FF" # symbols & pictographs
  u"\U0001F680-\U0001F6FF" # transport & map symbols
  u"\U0001F1E0-\U0001F1FF" # flags
  u"\U00002702-\U000027B0"
  u"\U000024C2-\U0001F251"
  "]+",flags=re.UNICODE
  )
  text = emoji_pattern.sub(r'',text)
  return text

In [19]:
#Remove URLS
def remove_http_links(text):
  text = re.sub('http://\S+|https://\S+','',text)
  return text

In [20]:
# Converting the contractions
def remove_contractions(text):
  text = contractions.fix(text)
  return text

In [21]:
# Remove stopwords
def remove_stopwords(text):
  text = " ".join([word for word in nltk.tokenize.word_tokenize(text) if word not in stopwords.words('english')])
  return text

In [22]:

# Lemmatization

nlp = spacy.load("en_core_web_sm", disable = ['parser', 'ner'])

def lemmatize_text(text):
  text = " ".join([token.lemma_ for token in nlp(text)])
  return text


C:\Users\gupta\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [23]:
#Remove Non-Alphabetic Characters
def discard_non_alpha(text):
  word_list_non_alpha = [word for word in regexp.tokenize(text) if word.isalpha()]
  text = " ".join(word_list_non_alpha)
  return text

In [24]:
# Aggregating all text processing functions
def text_clean(text):
  text = lower_text(text)
  text = remove_spaces(text)
  text = remove_punctuation(text)
  text = remove_html(text)
  text = remove_emoji(text)
  text = remove_http_links(text)
  text = remove_contractions(text)
  text = remove_stopwords(text)
  text = lemmatize_text(text)
  text = discard_non_alpha(text)
  # text = convert_abbrev(text)
  return text

In [25]:
# Initialize an empty dictionary to store the DataFrames
# Splitting the dataframe into parts
tweets_dfs = {}
for i in range(0,6):
  first = i*300000
  second = (i+1)*300000
  tweets_dfs[f"tweets_{i}"] = tweets.iloc[first:second,:]

In [26]:
tweets_dfs.keys()

dict_keys(['tweets_0', 'tweets_1', 'tweets_2', 'tweets_3', 'tweets_4', 'tweets_5'])

In [27]:
tweets_dfs['tweets_0']
tweets_dfs['tweets_1']
tweets_dfs['tweets_2']
tweets_dfs['tweets_3']
tweets_dfs['tweets_4']
tweets_dfs['tweets_5']

,target,text


In [28]:
for i in range(len(tweets_dfs.keys())):
  current_df = tweets_dfs[f"tweets_{i}"]
  current_df["Cleaned_Text"] = current_df["text"].apply(text_clean)

C:\Users\gupta\AppData\Local\Temp\ipykernel_26204\3162462804.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_df["Cleaned_Text"] = current_df["text"].apply(text_clean)


In [29]:
tweets_dfs['tweets_0']

,target,text,Cleaned_Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot awww bummer shoulda get david carr ...
1,0,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...
2,0,@Kenichan I dived many times for the ball. Man...,kenichan dive many time ball manage save rest ...
3,0,my whole body feels itchy and like its on fire,whole body feel itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",nationwideclass behave mad see
...,...,...,...
1497,0,@kcarruthers I'm only 540 years old in pixie y...,kcarruther year old pixie year
1498,0,body clock still up the chuffer..note never ev...,body clock still chuffernote never ever stay l...
1499,0,stupid thing wont let me get into my old twitt...,stupid thing let get old twitter page make new...
1500,0,I am soooo tired @ work,soooo tired work


In [30]:
# Merging the dataframes into a single dataframe
tweets_cleaned = pd.concat([tweets_dfs['tweets_0'],
tweets_dfs['tweets_1'],
tweets_dfs['tweets_2'],
tweets_dfs['tweets_3'],
tweets_dfs['tweets_4'],
tweets_dfs['tweets_5']],axis=0)

In [31]:
tweets_cleaned.head()

,target,text,Cleaned_Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot awww bummer shoulda get david carr ...
1,0,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...
2,0,@Kenichan I dived many times for the ball. Man...,kenichan dive many time ball manage save rest ...
3,0,my whole body feels itchy and like its on fire,whole body feel itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",nationwideclass behave mad see


In [32]:
tweets_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502 entries, 0 to 1501
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   target        1502 non-null   int64 
 1   text          1502 non-null   object
 2   Cleaned_Text  1502 non-null   object
dtypes: int64(1), object(2)
memory usage: 35.3+ KB


In [37]:
tweets_cleaned.to_csv('Clean_text.csv', index=False)

In [33]:
tweets_cleaned['target'] = tweets_cleaned['target'].replace(4,1)

In [34]:
num_labels = len(tweets_cleaned['target'].unique())
num_labels

1

In [35]:
tweets_cleaned['target'].value_counts()

0    1502
Name: target, dtype: int64

**Preprocessed Dataset**

In [40]:
cleaned = pd.read_csv("Clean_text.csv")
cleaned

,target,text,Cleaned_Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot awww bummer shoulda get david carr ...
1,0,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...
2,0,@Kenichan I dived many times for the ball. Man...,kenichan dive many time ball manage save rest ...
3,0,my whole body feels itchy and like its on fire,whole body feel itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",nationwideclass behave mad see
...,...,...,...
1497,0,@kcarruthers I'm only 540 years old in pixie y...,kcarruther year old pixie year
1498,0,body clock still up the chuffer..note never ev...,body clock still chuffernote never ever stay l...
1499,0,stupid thing wont let me get into my old twitt...,stupid thing let get old twitter page make new...
1500,0,I am soooo tired @ work,soooo tired work


In [41]:
cleaned.columns

Index(['target', 'text', 'Cleaned_Text'], dtype='object')

In [43]:
cleaned['target'] = cleaned['target'].replace(4,1)

In [44]:
# Splitting the dataframe into X and y and training and testing
from sklearn.model_selection import train_test_split
X = cleaned['Cleaned_Text']
y = cleaned["target"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train,dtype=np.float32)
y_test = np.array(y_test,dtype=np.float32)

In [45]:
X_train.shape

(1201,)

In [46]:
y_train.shape

(1201,)

In [47]:
y_test

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [48]:
# For training 1
batch_size = 4
max_words = 10000
max_len = 50
embedding_dim = 50
lstm_units = 32

In [49]:
# For training 2
batch_size = 8
max_sequence_length = 30
embedding_dim = 30
max_words = 25000
lstm_units = 32
max_len = 50

In [50]:
X_train = [str(x) for x in X_train]
X_test = [str(x) for x in X_test]
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
seqs = tok.texts_to_sequences(X_train)
test_seqs = tok.texts_to_sequences(X_test)
seq_matrix = sequence.pad_sequences(seqs,max_len)
test_seq_matrix = sequence.pad_sequences(test_seqs,max_len)

In [51]:
seq_matrix.shape

(1201, 50)

In [52]:
seq_matrix_dict = {}
for i in range(1,7):
  first = i*200000
  second = (i+1)*200000
  seq_matrix_dict[f"seq_matrix_{i}"] = seq_matrix[first:second,:]

In [53]:
seq_matrix_dict['seq_matrix_6'].shape
for i in range(1,len(seq_matrix_dict.keys())+1):
  print(seq_matrix_dict[f'seq_matrix_{i}'].shape)

(0, 50)
(0, 50)
(0, 50)
(0, 50)
(0, 50)
(0, 50)


In [54]:
# LSTM model
def LSTM_Model(words,len,dim,units,num_labels):
  model = tf.keras.Sequential()
  model.add(Embedding(input_dim=words,output_dim=dim,input_length=len))
  model.add(LSTM(units=units,dropout=0.2,recurrent_dropout=0.2))
  model.add(Dense(num_labels,activation='softmax'))

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [55]:
model = LSTM_Model(max_words,max_len,embedding_dim,lstm_units,num_labels)

In [56]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 30)            750000    
                                                                 
 lstm (LSTM)                 (None, 32)                8064      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 758097 (2.89 MB)
Trainable params: 758097 (2.89 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [57]:
model.fit(seq_matrix,y_train,epochs=3,batch_size=batch_size,validation_split=0.2,verbose=True)

Epoch 1/3
120/120 [==============================] - 14s 80ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
120/120 [==============================] - 8s 70ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/3
120/120 [==============================] - 7s 59ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00


In [58]:
prediction = model.predict(test_seq_matrix)
prediction.shape

10/10 [==============================] - 1s 10ms/step


(301, 1)